In [ ]:
import numpy as np
from gurobipy import *
import time

In [ ]:
# import SCNetworkData as data
import Deterministic as Det
S = Det.SSet
P = Det.PSet
C = Det.CSet
K = Det.KSet
N = Det.NSet
A = Det.ASet
C_f = Det.C_f
Q = Det.Q
D = Det.D
SP = Det.SP
M = Det.M
R = Det.R

In [ ]:
nS = len(S)
nP = len(P)
nC = len(C)
nK = len(K)

In [ ]:
BigSP = sum(SP.values()) # Supply Big M
BigD= sum(D.values()) # Demand Big M
print(D.values())
print(BigSP)
print(BigD)

In [ ]:
#Build Deterministic model for Supply Chain Cost Minimization
md = Model("Deterministic")

#Variable definitions
x = md.addVars(P, obj = C_f, vtype = GRB.BINARY, name = 'X')
y = md.addVars(A, K, obj = Q, name = 'Y')

md.modelSense = GRB.MINIMIZE
md.Params.outputFlag = 0  

#Constraints
# Logical constraints for product flow based on facility opening & customer 
md.addConstrs( (y.sum('*',j,k) <= BigSP * x[j] for j in P for k in K) , name = "Logical supply-side") #Big M constraints on RHS
md.addConstrs( (y.sum(j,'*',k) <= BigD * x[j] for j in P for k in K), name = "Logical demand-side") # Big M constraints on RHS

# Demand constraints
md.addConstrs( (y.sum('*',l,k) >= D[l,k] for l in C for k in K), name = "Demand")

# Flow balance constraints
md.addConstrs( (y.sum('*',j,k) == y.sum(j,'*',k) for j in P for k in K), name = "Flow Balance")

# Supply constraints
md.addConstrs( (y.sum(i,'*',k) <= SP[i,k] for i in S for k in K), name = "Supply Constraints")

# Processing constraints
md.addConstrs( (y.sum('*',j,k) * R[j,k] <= M[j] for j in P for k in K), name = "Processing Constraints" )

# Model Update
md.update()
# Model Solve 
md.optimize()

DetObj = md.objval
print("\n MP Obj: %g" % DetObj)
print("\n Runtime: ", str(md.Runtime))

In [ ]:
print((D.values()))

In [ ]:
for j in P:
    print(x[j].x)
for k in K:
    for (i,j) in A:
        print(i,j,k,y[(i,j,k)].x)

In [ ]:
# Parameters for Stochastic Extensive Form
import Stochastic as stoc
Scen = stoc.Scen
S = stoc.SSet
P = stoc.PSet
C = stoc.CSet
K = stoc.KSet
N = stoc.NSet
A = stoc.ASet
C_f = stoc.C_f
Q = stoc.Q
D = stoc.D
SP = stoc.SP
M = stoc.M
R = stoc.R

In [ ]:
nS = len(S)
nP = len(P)
nC = len(C)
nK = len(K)
nScen = len(Scen)
p_s = 1/nScen #Since all scenarios are equally likely
# Penalization factor
rho = 1000
Q_Scen = {}
for k in K:
    for (i,j) in A:
        Q_Scen[(i,j)] = p_s * Q[(i,j),k]
rho_Scen = p_s * rho

In [ ]:
# Scenario-wise Big M constraints for supply & demand
BigSP = {}
BigD = {}
for s in Scen:    
    BigSP[s] = (sum(SP[(i,k,s)] for i in S for k in K ))
    BigD[s] = (sum(D[(l,k,s)] for l in C for k in K ))

In [ ]:
# Build extensive form for Stochastic Solution

mse = Model("Stochastic Extensive Form")

# Variable definitions
x = mse.addVars(P, obj = C_f, vtype = GRB.BINARY, name = 'X') #Here & Now variable - configuration decision
y = mse.addVars(A, K, Scen, obj = Q_Scen, name = 'Y') #Recourse variable  - routing decisions

z = mse.addVars(C, K, Scen, obj = rho_Scen, name = 'slack' )

mse.modelSense = GRB.MINIMIZE
mse.Params.outputFlag = 0  

#Constraints
# Logical constraints for product flow based on facility opening & customer 
mse.addConstrs( (y.sum('*',j,k,s) <= BigSP[s] * x[j] for j in P for k in K for s in Scen) , name = "Logical supply-side")
mse.addConstrs( (y.sum(j,'*',k,s) <= BigD[s] * x[j] for j in P for k in K for s in Scen), name = "Logical demand-side")

# Demand constraints
mse.addConstrs( (y.sum('*',l,k,s) + z[l,k,s] == D[l,k,s] for l in C for k in K for s in Scen), name = "Demand")

# Flow balance constraints
mse.addConstrs( (y.sum('*',j,k,s) == y.sum(j,'*',k,s) for j in P for k in K for s in Scen), name = "Flow Balance")

# Supply constraints
mse.addConstrs( (y.sum(i,'*',k,s) <= SP[i,k,s] for i in S for k in K for s in Scen), name = "Supply Constraints")

# Processing constraints
mse.addConstrs( (sum( y.sum('*',j,k,s) * R[j,k] for k in K ) <= M[j,s] for j in P for s in Scen), name = "Processing Constraints" )

In [ ]:
mse.update()
mse.optimize()

In [ ]:
print(mse.objVal)
print("\n Runtime: ", str(mse.Runtime))

In [ ]:
for j in P:    
    print(x[j].x)
for s in Scen:
    for k in K:
        for (i,j) in A:
            print(i,j,k,s,y[(i,j,k,s)].x)

In [ ]:
# Benders Decomposition algorithm 
ViolTolerance = 0.001
BenderChoice = 1 # 0 for Multi-Cut, 1 for Single-Cut Benders
TrustDel = 1

In [ ]:
def ModifyAndSolveSP(s, xSol):
    #Modify RHS of constraints
    
    for k in K:
        for j in P:
            LogicSupplyConstrs[j,k].rhs = BigSP[s] * xSol[j] 
            LogicDemandConstrs[j,k].rhs = BigD[s] * xSol[j]
    
    for l in C: 
        for k in K:
            DemandConstrs[l,k].rhs = D[l,k,s]
    
    for i in S: 
        for k in K:    
            SupplyCapConstrs[i,k].rhs = SP[i,k,s]
            
    for j in P:
        ProcessConstrs[j].rhs = M[j,s] * xSol[j]
    
    SubP.update()
    SubP.optimize()
    
    SubPobj = SubP.objVal
    
    #Get Dual solution
    alphaSol = {}
    betaSol = {}
    gammaSol = {}
    thetaSol = {}
    psiSol = {}
    phiSol = {}
    
    for k in K:
        for j in P:
            alphaSol[j,k] = LogicSupplyConstrs[j,k].Pi
            betaSol[j,k] = LogicDemandConstrs[j,k].Pi
    
    for k in K:
        for l in C:
            gammaSol[l,k] = DemandConstrs[l,k].Pi
            
    for k in K:
        for j in P:
            psiSol[j,k] = FlowBalanceConstrs[j,k].Pi
        
    for k in K:        
        for i in S:
            phiSol[i,k] = SupplyCapConstrs[i,k].Pi
            
    for j in P:
            thetaSol[j] = ProcessConstrs[j].Pi
        
    if BenderChoice == 0:
        CutFound = False
        if(etaSol[s] < SubPobj - ViolTolerance): # Found Benders cut is violated at the current master solution
            CutFound = True

        return SubPobj, CutFound, alphaSol, betaSol, gammaSol, thetaSol, psiSol, phiSol
    
    else:
        CutFound  = True
        return SubPobj, CutFound, alphaSol, betaSol, gammaSol, thetaSol, psiSol, phiSol

In [ ]:
#Build Master problem
if BenderChoice == 0:
    
    MP = Model("Master Problem - MultiCut")

    #Variable definitions
    x = MP.addVars(P, obj = C_f, vtype = GRB.BINARY, name = 'x')
    eta = MP.addVars(Scen, obj = p_s, name = 'eta')

    MP.Params.outputFlag = 0  # turn off output
    MP.Params.method = 1      # dual simplex
    MP.Params.logtoconsole = 0

    MP.modelSense = GRB.MINIMIZE
else:
    MP = Model("Master Problem - Single Cut")
    
    #Variable definitions
    x = MP.addVars(P, obj = C_f, vtype = GRB.BINARY, name = 'x')
    thetaMaster = MP.addVar(obj = 1, name = 'theta')
    
    MP.Params.outputFlag = 0  # turn off output
    MP.Params.method = 1      # dual simplex
    MP.Params.logtoconsole = 0

    MP.modelSense = GRB.MINIMIZE

In [ ]:
#Build Subproblem Primal

SubP = Model("Subproblem")

#Variable definitions
y = SubP.addVars(A, K, obj = Q, name = 'Y')
z = SubP.addVars(C, K, obj = rho, name = 'slack' )

SubP.modelSense = GRB.MINIMIZE
SubP.Params.outputFlag = 0 

#Constraint Dictionaries
LogicSupplyConstrs = {}
LogicDemandConstrs = {}
DemandConstrs = {}
SupplyCapConstrs = {}
ProcessConstrs = {}
FlowBalanceConstrs = {}

for k in K:
    for j in P: 
        LogicSupplyConstrs[j,k] = SubP.addConstr( y.sum('*',j,k) <= 0  , name = "Logical supply-side") #xsol needs assignment
        LogicDemandConstrs[j,k] = SubP.addConstr( y.sum(j,'*',k) <= 0 , name = "Logical demand-side") #xsol needs assignment

for k in K:
    for l in C: 
        DemandConstrs[l,k] = SubP.addConstr( y.sum('*',l,k) + z[l,k] == 0 , name = "Demand") #RHS to be updated for each scenario

for k in K:
    for j in P: 
        FlowBalanceConstrs[j,k] = SubP.addConstr( (y.sum('*',j,k) == y.sum(j,'*',k) ), name = "Flow Balance")

for k in K:        
    for i in S:    
        SupplyCapConstrs[i,k] = SubP.addConstr( y.sum(i,'*',k) <= 0 , name = "Supply Constraints") #RHS to be updated for each scenario

for j in P:
    ProcessConstrs[j] = SubP.addConstr( sum( y.sum('*',j,k) * R[j,k] for k in K ) <= 0, name = "Processing Constraints" ) #RHS to be updated for each scenario

In [ ]:
#Benders Cut Generation Algorithm
start = time.time()
if BenderChoice == 0:
    cutFound = True
    N_Iters = 0
    BestUB = GRB.INFINITY
    
    while(cutFound & N_Iters <= 200):
        
        N_Iters +=1
        cutFound = False
        
        #Solve MP
        MP.update()
        MP.optimize()
        
        #Get solution
        MPobj = MP.objval
        print("MP Obj: %g" % MPobj)
        
        xSol ={}
        for j in P:
            xSol[j] = x[j].x
        etaSol = {}
        for s in Scen:
            etaSol[s] = eta[s].x
            
        UB = 0
        for j in P:
            UB += C_f[j] * xSol[j]
            
        for s in Scen:
            QVal, cutFound_s, alpha, beta, gamma, theta, psi, phi = ModifyAndSolveSP(s, xSol)
            
            UB += p_s * QVal
            
            if(cutFound_s):
                CutFound = True
                expr = LinExpr(eta[s] - (sum(BigSP[s] * x[j] * alpha[j,k] for j in P for k in K) + sum( BigD[s] * x[j] * beta[j,k] for j in P for k in K)
                               +sum( D[l,k,s] * gamma[l,k] for l in C for k in K ) + sum( M[j,s] * x[j] * theta[j] for j in P ) + 
                                         sum( SP[i,k,s] * phi[i,k] for i in S for k in K ) ))
                MP.addConstr(expr >= 0)
                #print("CUT: " + str(expr) + " >= 0")
                
        if (UB < BestUB):
            BestUB = UB
        
        print("LB: " +str(MPobj) )
        print("UB: " + str(UB) )
        print("BestUB: " + str(BestUB) + "\n")
        
        if (UB - MPobj <= ViolTolerance):
            break
    
    end = time.time()  
        
    
    print('\nOptimal Solution:')
    print('MPobj: %g' % MPobj)
    print("x solution: " + str(xSol))
    print("eta solution: " + str(etaSol))
    print("NoIters: " + str(N_Iters))    
    print("\n Runtime: ", (end - start))    
if BenderChoice == 1:
    cutFound = True
    N_Iters = 0
    BestUB = GRB.INFINITY
    
    while(N_Iters <= 1200):  #Change Iter limit
        
        N_Iters +=  1
        cutFound = False
        
        #Solve MP
        MP.update()
        MP.optimize()
        
        MPobj = MP.objVal
        
        xSol ={}
        for j in P:
            xSol[j] = x[j].x
            
        thetaMasterSol = thetaMaster.x
        
        LB = 0
        for j in P:
            LB += C_f[j] * xSol[j]
        LB += thetaMasterSol
        
        QS = 0
        expr = 0
        for s in Scen:
            QVal, cutFound_s, alpha, beta, gamma, theta, psi, phi = ModifyAndSolveSP(s, xSol)
            if(cutFound_s):
                CutFound = True
                expr += LinExpr(p_s*(sum(BigSP[s] * x[j] * alpha[j,k] for j in P for k in K) + sum( BigD[s] * x[j] * beta[j,k] for j in P for k in K)
                               +sum( D[l,k,s] * gamma[l,k] for l in C for k in K ) + sum( M[j,s] * x[j] * theta[j] for j in P ) + 
                                         sum( SP[i,k,s] * phi[i,k] for i in S for k in K ) ))
                QS += p_s * QVal
        
        UB = 0
        for j in P:
            UB += C_f[j] * xSol[j]
        UB += QS
        
        if (thetaMasterSol - QS < ViolTolerance ):
            cutFound = True
            MP.addConstr(thetaMaster - expr  >= 0)
            #print("CUT: " + str(thetaMaster - expr) + " >= 0")
        else:
            break
            
        if (UB < BestUB):
            BestUB = UB
            
        if (UB - MPobj <= ViolTolerance):
            break
            
        print("LB: "+str(LB) + "\n")
        print("UB: " + str(UB) + "\n")
        print("BestUB: " + str(BestUB) + "\n")
        
    end = time.time()  
        
    print('\nOptimal Solution:')
    print('MPobj: %g' % MPobj)
    print("x sol: " + str(xSol))
    print("theta Master sol: " + str(thetaMasterSol))
    print("NoIters: " + str(N_Iters))   
    print("\n Runtime: ", (end - start))

In [ ]:
# Define SubProblem Dual

SPD = Model("Sub Problem Dual")

#Define Variables

alphaVar = SPD.addVars(P, K, ub = 0, lb = -1e20, name = "alpha"  )
betaVar = SPD.addVars(P, K, ub = 0, lb = -1e20, name  = "beta" )
gammaVar = SPD.addVars(C, K, ub = rho, lb = 0, name = "gamma" )
thetaVar = SPD.addVars(P, ub = 0, lb = -1e20, name = "theta")
psiVar = SPD.addVars(P, K, lb = -1e20, name = "psi" )
phiVar = SPD.addVars(S, K, lb = -1e20, ub = 0, name ="phi")

#Define constraints
for k in K:
    for i in S:
        for j in P:
            SPD.addConstr( alphaVar[j,k] + phiVar[i,k] + R[j,k] * thetaVar[j] + psiVar[j,k] <= Q[(i,j),k] )
            
for k in K:
    for j in P:
        for l in C:
            SPD.addConstr( betaVar[j,k] + gammaVar[l,k] - psiVar[j,k] <= Q[(j,l),k] )

# One more constraint to be added : Sub-problem equality constraint 

SPD.Params.outputFlag = 0  # turn off output
SPD.Params.method = 1      # dual simplex
SPD.Params.logtoconsole = 0
SPD.modelSense = GRB.MAXIMIZE

In [ ]:
# Solve modified dual with core-point to get Pareto Optimal cut
def SolveDualWithCorePoint(x0, xSol, s, QVal):
    
    SPD.setObjective( sum(BigSP[s] * x0[j] * alphaVar[j,k] for j in P for k in K) + sum(BigD[s] * x0[j] * betaVar[j,k] for j in P for k in K) +
                    sum(D[l,k,s] * gammaVar[l,k] for l in C for k in K) + sum(M[j,s] * x0[j] * thetaVar[j] for j in P) + 
                     sum(SP[i,k,s] * phiVar[i,k] for i in S for k in K) )
    
    #Adding constraint that restricts solutions to dual to feasible subproblem solutions
    SPD.addConstr( sum(BigSP[s] * xSol[j] * alphaVar[j,k] for j in P for k in K) + sum(BigD[s] * xSol[j] * betaVar[j,k] for j in P for k in K) +
                    sum(D[l,k,s] * gammaVar[l,k] for l in C for k in K) + sum(M[j,s] * xSol[j] * thetaVar[j] for j in P) + 
                     sum(SP[i,k,s] * phiVar[i,k] for i in S for k in K) == QVal, name = "solution constraint")
    
    SPD.update()
    SPD.optimize()
    
    
    SPDVal = SPD.objVal
    
    alphaVarSol = {}
    for k in K:
        for j in P:
            alphaVarSol[j,k] = alphaVar[j,k].x
    
    betaVarSol = {}
    for k in K:
        for j in P:
            betaVarSol[j,k] = betaVar[j,k].x
    
    gammaVarSol = {}
    for k in K:
        for l in C:
            gammaVarSol[l,k] = gammaVar[l,k].x
            
    thetaVarSol = {}
    for j in P:
        thetaVarSol[j] = thetaVar[j].x
        
    psiVarSol = {}
    for k in K:
        for j in P:
            psiVarSol[j,k] = psiVar[j,k].x
            
    phiVarSol = {}
    for k in K:
        for i in S:
            phiVarSol[i,k] = phiVar[i,k].x
    
    SPD.remove(SPD.getConstrByName("solution constraint"))
    SPD.update()
    
    return SPDVal, alphaVarSol, betaVarSol, gammaVarSol, thetaVarSol, psiVarSol, phiVarSol

In [ ]:
# Getting Pareto Optimal Cuts
def FindParetoOptimalCut(x0, xSol):
    
    QS = 0
    expr = 0
    
    for s in Scen:
            
        QVal, cutFound_s, alpha, beta, gamma, theta, psi, phi = ModifyAndSolveSP(s, xSol)
            
        #Solve modified dual with identified core point 
        SPDVal, alphaVarSol, betaVarSol, gammaVarSol, thetaVarSol, psiVarSol, phiVarSol = SolveDualWithCorePoint(x0, xSol, s, QVal)
        
        #Collect this scenario's Pareto Optimal cut in expr
        expr += LinExpr(p_s*(sum(BigSP[s] * x[j] * alphaVarSol[j,k] for j in P for k in K) + sum( BigD[s] * x[j] * betaVarSol[j,k] for j in P for k in K)
                               +sum( D[l,k,s] * gammaVarSol[l,k] for l in C for k in K ) + sum( M[j,s] * x[j] * thetaVarSol[j] for j in P ) + 
                                         sum( SP[i,k,s] * phiVarSol[i,k] for i in S for k in K ) ))
        QS += p_s * QVal
    
    return expr, QS

In [ ]:
# Accelerated Benders Decomposition Algorithm
start = time.time()
cutFound = True
N_Iters = 0
BestUB = GRB.INFINITY
HammingConstraint = False
    
while(N_Iters <= 1200):  #Change Iter limit
        
    N_Iters +=  1
    cutFound = False

    #Solve MP
    MP.update()
    MP.optimize()

    MPobj = MP.objVal

    xSol ={}
    for j in P:
        xSol[j] = x[j].x

    thetaMasterSol = thetaMaster.x

    LB = 0
    for j in P:
        LB += C_f[j] * xSol[j]
    LB += thetaMasterSol
    
#     #Remove Hamming distance constraint
#     if(HammingConstraint):    
#         MP.remove(MP.getConstrByName("Hamming Dist"))
#         MP.update()
    
    QS = 0
    expr = 0
    
    #Valid Inequality terms
    a = {}
    b = 0
    for j in P:
         a[j] = 0
    
    #Logistics constraints terms
    D_sum = {}
    M_sum = {}
    z_sum = {}
    
    for j in P:
         M_sum[j] = 0
        
    for k in K:
        for l in C:
            z_sum[l,k] = 0
            D_sum[l,k] = 0
    
    for s in Scen:
        QVal, cutFound_s, alpha, beta, gamma, theta, psi, phi = ModifyAndSolveSP(s, xSol)
        if(cutFound_s):
            CutFound = True
            expr += LinExpr(p_s*(sum(BigSP[s] * x[j] * alpha[j,k] for j in P for k in K) + sum( BigD[s] * x[j] * beta[j,k] for j in P for k in K)
                           +sum( D[l,k,s] * gamma[l,k] for l in C for k in K ) + sum( M[j,s] * x[j] * theta[j] for j in P ) + 
                                     sum( SP[i,k,s] * phi[i,k] for i in S for k in K ) ))
            # Towards constructing Knapsack Inequality 
            for j in P:
                a[j] += p_s*(sum(BigSP[s] * alpha[j,k] for k in K) + sum( BigD[s] * beta[j,k] for k in K) +  M[j,s] * theta[j])
            b += p_s*(sum( D[l,k,s] * gamma[l,k] for l in C for k in K ) + sum( SP[i,k,s] * phi[i,k] for i in S for k in K ))
            QS += p_s * QVal
            
            # Towards constructing Logistics Constraints
            for j in P:
                M_sum[j] += M[j,s]
                
            for k in K:
                for l in C:
                    z_sum[l,k] += z[l,k].x
                    D_sum[l,k] += D[l,k,s]
    UB = 0
    for j in P:
        UB += C_f[j] * xSol[j]
    UB += QS
    
    if (UB < BestUB):
        BestUB = UB
     
    # Regular cut generation
    if (thetaMasterSol - QS < ViolTolerance ):
        cutFound = True
        MP.addConstr(thetaMaster - expr  >= 0)   #optimality cut
        #print("CUT: " + str(thetaMaster - expr) + " >= 0")
                  
    else:
        break
    
    
    LHC = 0
    for j in P:
        LHC += math.floor( C_f[j] + a[j] ) * x[j]
        RHC = math.floor( BestUB - b )
    
    #Adding Knapsack Inequality
    MP.addConstr(LHC <= RHC)  
        
    M_avg = {}
    D_avg = {}
    z_avg = {}
        
    for j in P:
        M_avg[j] = M_sum[j]/nScen
        
    for k in K:
        for l in C:
            D_avg[l,k] = D_sum[l,k]/nScen
            z_avg[l,k] = z_sum[l,k]/nScen
        
    #Adding Logistics constraints
#     for k in K:
#         for l in C:
#             MP.addConstr( sum( (M_avg[j]/R[j,k]) * x[j] for j in P ) + z_avg[l,k] >= D_avg[l,k] )  
    
    #Cut Strengthening 
    
    #Solve Master Problem Relaxation
    for j in P:
        x[j].vtype = GRB.CONTINUOUS
    
    MP.update()
    MP.optimize()
    
    #Picking core point as fractional optimal solution of LP relaxation
    x0 = {}
    for j in P:
        x0[j] = x[j].x
        
    #Reinstate integrality restriction to Master
    for j in P:
        x[j].vtype = GRB.BINARY
    MP.update()
        
    #Pass core point & integer feasible solution to find Pareto Optimal cut
    ParetoCutExpr , QS = FindParetoOptimalCut(x0, xSol)
    
    UB = 0
    for j in P:
        UB += C_f[j] * xSol[j]
    UB += QS
    
    #Adding Pareto Optimal Cut
#     if( thetaMasterSol - QS <= ViolTolerance ):
#         cutFound = True
#         MP.addConstr(thetaMaster - ParetoCutExpr  >= 0)
#         #print("CUT: " + str(thetaMaster - ParetoCutExpr) + " >= 0")
    
    MP.update()

    print("LB: "+str(LB) + "\n")
    print("UB: " + str(UB) + "\n")
    print("BestUB: " + str(BestUB) + "\n")
    
    if (UB - MPobj <= ViolTolerance):
        break
    
#     HamSet = []
#     DiffSet = []
#     for j in P:  
#         if (xSol[j] == 1):
#             HamSet.append(j)
#         else:
#             DiffSet.append(j)
    
#     MP.update()
#     #Hamming Distance constraint
    
#     if(N_Iters <= 5):
#         MP.addConstr( sum((1-x[j]) for j in HamSet) + sum(x[j] for j in DiffSet) <= TrustDel, "Hamming Dist" )
#         HammingConstraint = True
#     else:
#         HammingConstraint = False
        
    if(UB <= BestUB):
         BestUB = UB
     

end = time.time()    
print('\nOptimal Solution:')
print('MPobj: %g' % MPobj)
print("x sol: " + str(xSol))
print("theta Master sol: " + str(thetaMasterSol))
print("NoIters: " + str(N_Iters))
print("\n Runtime: ", str(MP.Runtime))
print("Time taken",(end - start))

In [ ]:
#For a given candidate solution x, Oracle provides function value and a valid sub-gradient as OP
def SPOracle(w):
    
    objVals = []
    
    subgradTerm = {}
    for j in P:
        subgradTerm[j] = 0 
    
    
    for s in Scen:
        
        QVal, cutFound_s, alpha, beta, gamma, theta, psi, phi = ModifyAndSolveSP(s, w)
        
        T_alpha = {}
        T_beta = {}
        T_theta = {}
        for k in K:
            for j in P:
                T_alpha[j,k] = -BigSP[s]
                T_beta[j,k] = -BigD[s]
            
        for j in P:
            T_theta[j] = -M[j,s]
        
        objVals.append(QVal)
        for j in P:
            subgradTerm[j] += p_s * ( sum(T_alpha[j,k] * alpha[j,k] + T_beta[j,k] * beta[j,k] for k in K) + 
                                     T_theta[j] * theta[j] )  
            
    funcVal = sum(C_f[j] * w[j] for j in P) + p_s * sum(objVals)
    
    subgradient = {}
    for j in P:
        subgradient[j] = C_f[j] - subgradTerm[j]
    
    return funcVal, subgradient

In [ ]:
# Level Method Implementation
lamda = 0.3
IterCount = 0
CutCount = 0
x_t = {}
for j in P:
    x_t[j] = 0

In [ ]:
# Build Master Problem
m = Model("Level Master")

#Variable definitions
x = m.addVars(P, vtype = GRB.BINARY, name = 'x')
f_t = m.addVar(name = 'Function at iteration t' )

m.Params.outputFlag = 0  # turn off output
m.Params.method = 1      # dual simplex
m.Params.logtoconsole = 0

In [ ]:
# Level Method Implementation

funcVals = []
subgrads = []

checker = True

while checker:
    
    #Compute function value and subgradient at x_t
    funcVal, subgrad = SPOracle(x_t)
    
    #Store values computed into array s
    funcVals.append(funcVal)
    subgrads.append(subgrad)
    
    Expr = 0
    for j in P:
        Expr += subgrad[j] * (x[j] - x_t[j])
    
    m.addConstr( f_t >= funcVal + Expr , name = 'Cut')
    CutCount += 1
    
    print("Cut:",str(f_t),">=",str(funcVal),"+",str(Expr))
    
    #Reset objective to master problem
    m.setObjective( f_t , GRB.MINIMIZE)
    m.update()
    
    m.optimize()
    
    z_LB = m.objVal 
    #print(z_LB)
    z_UB = min(funcVals)
    #print(z_UB)
    
    #Project
    Level_t = z_LB + lamda * ( z_UB - z_LB )
    
    #Update model to setup projection problem
    m.update()
    
    l2NormSq = sum( (x[j] - x_t[j]) * (x[j] - x_t[j]) for j in P )
    
    #Set new objective for projection
    m.setObjective( QuadExpr(l2NormSq) , GRB.MINIMIZE)
    #Add level constraint
    m.addConstr( f_t <= Level_t, "Level" )
    # Solve projection problem
    m.update()
    m.optimize()
    
     
    # Set next value of x for projection
    for j in P:
        x_t[j] = x[j].x
        
    IterCount += 1
    
    #Remove level constraint outside of projection problem
    m.remove(m.getConstrByName("Level"))
    m.update()
    
    print("At iteration:",str(IterCount))
    print("Best Lower Bound:",str(z_LB))
    print("Best Upper Bound:",str(z_UB))
    print("Optimality Gap:",str(z_UB - z_LB))
    
    if(z_UB - z_LB <= ViolTolerance):
        break

print("Total number of iterations:",str(IterCount))
print("Total number of cuts:",str(CutCount))
print("Solution time",m.runtime)